<a href="https://colab.research.google.com/github/ti-a-go/analise_de_dados_tweeter/blob/main/Extraindo_tweets_utilizando_snscrape_(NER).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Preparando o [Spacy](https://spacy.io/)

In [ ]:
!pip install -U pip setuptools wheel

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download pt_core_news_lg

In [ ]:
!python -m spacy download en_core_web_trf

In [5]:
import spacy

In [6]:
spacy

<module 'spacy' from '/usr/local/lib/python3.7/dist-packages/spacy/__init__.py'>

# 2. Buscando dados no Tweeter usando o `snscrape`

In [ ]:
!pip3 install snscrape

In [8]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

**Input: `query`, `maxTweets`**

`query` será utilizada como input para a busca no twiter

`maxTweets` será a quantidade de maxima de twits buscada


In [9]:
query = 'Simone Tebet since:2022-10-02 until:2022-10-07'
maxTweets = 100

Busca uma quantidade de `maxTweets` utilizando a  `query`

In [10]:
i = 0
tweets_list = []
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
  if i > maxTweets:
    break
  tweets_list.append([tweet.date, tweet.url, tweet.username, tweet.content])
  i = i + 1

Resultado: lista de twites

In [ ]:
tweets_list

Dataframe: tweets

In [12]:
tweets_df = pd.DataFrame(tweets_list, columns=['date', 'url','username', 'content' ])
tweets_df

,date,url,username,content
0,2022-10-06 23:59:56+00:00,https://twitter.com/amadouaminata/status/15781...,amadouaminata,"@simonetebetbr @LulaOficial Obrigado, Presiden..."
1,2022-10-06 23:59:43+00:00,https://twitter.com/MARCELOCALDASP1/status/157...,MARCELOCALDASP1,@JosueLucca @KaykyStt @nahccas @manapg01 @Mate...
2,2022-10-06 23:59:39+00:00,https://twitter.com/marlimpoletto/status/15781...,marlimpoletto,@simonetebetbr Vc foi uma piada.
3,2022-10-06 23:59:37+00:00,https://twitter.com/DaysonCamargoM/status/1578...,DaysonCamargoM,@simonetebetbr Quem te viu quem te vê! \nHoje ...
4,2022-10-06 23:59:34+00:00,https://twitter.com/prjairomaciel/status/15781...,prjairomaciel,"@simonetebetbr É claro fia, tu quer uma tetona..."
...,...,...,...,...
96,2022-10-06 23:52:34+00:00,https://twitter.com/LiliaFeBSB/status/15781713...,LiliaFeBSB,@GLAUBERGCO @augustodefranco @simonetebetbr Nã...
97,2022-10-06 23:52:29+00:00,https://twitter.com/Emosglobina/status/1578171...,Emosglobina,"Não aguento, não aguento a galera falando nem ..."
98,2022-10-06 23:52:29+00:00,https://twitter.com/NiltonC76472742/status/157...,NiltonC76472742,@simonetebetbr Vc não tem caráter mesmo.
99,2022-10-06 23:52:28+00:00,https://twitter.com/BastardaInglri1/status/157...,BastardaInglri1,"@simonetebetbr @LulaOficial Parabéns, foi a me..."


Podemos salvar o dataframe tweets num arquivo `.csv`

In [13]:
# tweets_df.to_csv()

# 3. Carregando o modelo pré treinado da língua portugesa (Spacy)

O [Spacy](https://spacy.io/models) já nos dá dezenas de modelos pré treinados de várias línguas. Podemos carregar um modelo específico (português, por exemplo) como uma função e vamos usar essa função para processar os tweets extraídos

In [15]:
# Carregamos um modelo executando a função spacy.load("nome_do_model")
pt_model = spacy.load("pt_core_news_lg")

# Vamos carregar também um modelo da língua inglêsa
en_model = spacy.load("en_core_web_trf")

pt_model

In [16]:
en_model

## 3.1 Introdução ao Spacy (Doc e Token)

In [17]:
# vamos processar o conteúdo do primeiro tweet e criar um objeto chamado Documento (Doc)
doc = pt_model(tweets_df["content"][0])

# veja que o documento parece com uma string, mas é uma lista de tokens
doc

@simonetebetbr @LulaOficial Obrigado, Presidente Simone. O Brasil precisa de pessoas como a Senhora para tornar este pais, pais de todo o mundo. Um Pais rico como o Brasil, minha segunda Patria, não pode ter tantas pessoas pobres e miseraveis.

O [Documento](https://spacy.io/api/doc) é um conceito chave para o Spacy. Recomento ler a documentação

In [18]:
type(doc)

spacy.tokens.doc.Doc

O [Token](https://spacy.io/api/token) é o conceito central do Spacy. E no token que vamos encontrar todas a anotações linguísticas feitas pelos modelo pré treinado que utilizamos para processar o texto do tweet

In [ ]:
# podemos iterar pelos token de um doc
for token in doc:
  print(f"Token: {token}")

  # o token tem vários atributos que correspondem às lebels atribuídas pelo modelo
  # um deles é o "pos_" que representa a classe da palavra
  print(f"  POS: {token.pos_}\n")

In [20]:
# vemos que o segundo token é um adverb (abvérbio)
doc[1].pos_

'ADJ'

o Spacy utiliza [um glossário](https://github.com/explosion/spaCy/blob/master/spacy/glossary.py) com as explicações para as abreviações utilizadas pela biblioteca

In [21]:
# podemos acessar o glossário a partir da função spacy.explain(sigla)
spacy.explain(doc[1].pos_)

'adjective'

# 4. Início da Análise (NER)

## Entidade Nomeada

O Doc é mais do que uma lista, é uma classe que possui um atributo chamado `ents` que também é uma lista de tokens, mas só aqueles que o modelo "entende" como sendo uma [entidade nomeada](https://pt.wikipedia.org/wiki/Reconhecimento_de_entidade_mencionada).

In [22]:
"""
   No primeiro tweet o modelo encontrou uma entidade (Simone Tebet)
   que é justamente o valor da nossa query
"""
doc.ents

(@simonetebetbr, Brasil, Senhora, Pais, Brasil)

Uma **entidade nomeada** possui uma label anotada pelo modelo

In [23]:
for ent in doc.ents:
  print(f"token: {ent.text}")
  print(f"  NER: {ent.label_}")

token: @simonetebetbr
  NER: MISC
token: Brasil
  NER: LOC
token: Senhora
  NER: PER
token: Pais
  NER: PER
token: Brasil
  NER: LOC


In [24]:
# Named person or family (Nome de pessoa ou família)
spacy.explain("PER")

'Named person or family.'

## Entidades encontradas em todos os tweets

Até agora só processamos um tweet, vamos processar todos extraídos para podermos saber quais entidades nomeadas o spacy encontrou


In [ ]:
docs = []
for tweet in tweets_df["content"]:
  docs.append(pt_model(tweet))
docs

In [26]:
type(docs[0])

spacy.tokens.doc.Doc

In [27]:
docs[1].ents

(@JosueLucca, Arminio Fraga, Venezuela, Argentina)

In [ ]:
ents = [doc.ents for doc in docs]
ents

# 5. Wordcloud com entidades nomeadas encontradas pelo Spacy

In [ ]:
ents

In [ ]:
ents_for_word_cloud = []
for ent in ents:
  if len(ent):
    ents_for_word_cloud.append(ent[0].text)
ents_for_word_cloud

In [ ]:
ents_for_word_cloud = " ".join(ents_for_word_cloud)
ents_for_word_cloud

In [ ]:
!pip install wordcloud

In [54]:
from wordcloud import WordCloud
import os

In [53]:
wordcloud = WordCloud(width=2000, height=1000).generate(ents_for_word_cloud)

In [56]:
image_file = 'entity_cloud.png'
image = wordcloud.to_image()
image.save(image_file)

# 6. Anotações **Linguísticas** (linguistic features)

# 7. Acurácia desses modelos pré treinados